<a href="https://colab.research.google.com/github/MdelaVilla/MB/blob/main/NER_para_MB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load model directly
  from transformers import AutoTokenizer, AutoModelForTokenClassification
  from transformers import pipeline

  tokenizer = AutoTokenizer.from_pretrained("Clinical-AI-Apollo/Medical-NER")
  model = AutoModelForTokenClassification.from_pretrained("Clinical-AI-Apollo/Medical-NER")


In [ ]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "the investigations reported here offer a model for further research into possible intrauterine factors in the pathogenesis of hyaline membrane disease"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-DISEASE_DISORDER', 'score': 0.47977594, 'index': 19, 'word': '▁hy', 'start': 125, 'end': 128}, {'entity': 'I-DISEASE_DISORDER', 'score': 0.405778, 'index': 20, 'word': 'a', 'start': 128, 'end': 129}, {'entity': 'I-DISEASE_DISORDER', 'score': 0.45834148, 'index': 21, 'word': 'line', 'start': 129, 'end': 133}, {'entity': 'I-DISEASE_DISORDER', 'score': 0.5808652, 'index': 22, 'word': '▁membrane', 'start': 133, 'end': 142}, {'entity': 'I-DISEASE_DISORDER', 'score': 0.6104399, 'index': 23, 'word': '▁disease', 'start': 142, 'end': 150}]


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Cargar el modelo y el tokenizador preentrenados
tokenizer = AutoTokenizer.from_pretrained("Clinical-AI-Apollo/Medical-NER")
model = AutoModelForTokenClassification.from_pretrained("Clinical-AI-Apollo/Medical-NER")

# Crear un pipeline para el reconocimiento de entidades nombradas
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

def enrich_text_with_ner(text):
    # Obtener los resultados del reconocimiento de entidades nombradas
    ner_results = nlp(text)

    # Ordenar las entidades por sus posiciones de inicio
    ner_results = sorted(ner_results, key=lambda x: x['start'])

    # Inicializar una lista para almacenar las entidades combinadas
    combined_entities = []

    # Combinar entidades contiguas del mismo tipo
    for entity in ner_results:
        entity_type = entity['entity'].split('-')[-1]  # Obtener el tipo de entidad sin el prefijo B- o I-
        if combined_entities and entity_type == combined_entities[-1]['entity']:
            combined_entities[-1]['end'] = entity['end']
            combined_entities[-1]['word'] += text[entity['start']:entity['end']]
        else:
            combined_entities.append({
                'entity': entity_type,
                'start': entity['start'],
                'end': entity['end'],
                'word': text[entity['start']:entity['end']]
            })

    # Inicializar una cadena vacía para almacenar el texto enriquecido
    enriched_text = ""

    # Inicializar una variable para rastrear la última posición en el texto
    last_position = 0

    # Iterar sobre las entidades combinadas y enriquecer el texto
    for entity in combined_entities:
        # Agregar el texto antes de la entidad
        enriched_text += text[last_position:entity['start']]

        # Agregar la entidad con etiquetas
        enriched_text += f"<{entity['entity']}>{entity['word']}</{entity['entity']}>"

        # Actualizar la última posición
        last_position = entity['end']

    # Agregar el texto restante después de la última entidad
    enriched_text += text[last_position:]

    return enriched_text

# Texto de ejemplo
example_text = "the investigations reported here offer a model for further research into possible intrauterine factors in the pathogenesis of hyaline membrane disease"

# Enriquecer el texto con etiquetas de NER y mostrar el resultado
enriched_text = enrich_text_with_ner(example_text)
print(enriched_text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


the investigations reported here offer a model for further research into possible intrauterine factors in the pathogenesis of<DISEASE_DISORDER> hyaline membrane disease</DISEASE_DISORDER>
